# Training notebook

* This notebook is used for building, training and evaluating CNN models on MURA dataset.  
* For the purpose of experimentation, the CNN model, dataset type (original / preprocessed offline), online augmentation and preprocessing can be easily changed with the use of custom loading functions
* More information on used functions can be found mainly in python files ***utils.py*** and ***image_preprocessing.py***
* Some cells are specific for Google Colab environment and might not work correctly in other environments

In [1]:
%tensorflow_version 2.x
import tensorflow as tf
import pandas as pd
import pickle
import numpy as np
import cv2
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam

import matplotlib.pyplot as plt
%matplotlib inline

# Google Colab does not have tensorflow_addons installed by default
!pip install tensorflow-addons
from tensorflow_addons.metrics import CohenKappa

     |████████████████████████████████| 1.1 MB 8.6 MB/s 


# FILEPATHS


In [2]:
DRIVE_DIR = '/content/drive/MyDrive/MURA/' # Directory in my personal Google Drive with all source files and datasets
MURA_DIR = '/content/original/' # Directory with original MURA dataset
CLAHE_2_DIR = '/content/clahe_2/' # Directory for CLAHE preprocessed dataset with clipLimit=2
CLAHE_10_DIR = '/content/clahe_10/' # Directory for CLAHE preprocessed dataset with clipLimit=10
CROPPED_DIR = '/content/cropped/' # Directory for custom cropping preprocessed dataset
DATAFRAME_PATH = DRIVE_DIR + '/tvt_detailed_paths.csv' # Path to csv file with dataset information (train-valid-test split)
CHECKPOINT_DIR = '/content/drive/MyDrive/MURA/models/' # Root directory for storing models

# Google Colab specific section

## Mount Google Drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Google colab functions

Next cell containts function definitions, that are needed or useful when running notebook in Google Colab, such as copying dataset from Google Drive, checking GPU utilization, copying other files from Google Drive.

In [4]:
# This function is from Google Colab guide notebooks: https://colab.research.google.com/notebooks/pro.ipynb
def check_gpu():
  """
  Prints information about available GPU or message saying there isn't one
  """
  gpu_info = !nvidia-smi
  gpu_info = '\n'.join(gpu_info)
  if gpu_info.find('failed') >= 0:
    print('Not connected to a GPU')
  else:
    print(gpu_info)


def init_file_struct():
  """
  Prep file structure in Google Colab and copies neccessary files from Google Drive
  """
  !cp "{DRIVE_DIR}src/utils.py" .
  !cp "{DRIVE_DIR}src/image_preprocessing.py" .


def copy_dataset(filename):
  """
  Copies and unzips dataset from Google Drive and deletes the zipped file

  Parameters
  ----------
  filename : str
    Dataset filename, without its path, the path will be added from constants
  """
  !cp {DRIVE_DIR}{filename} /content/

  !unzip -q /content/{filename}

  !rm /content/{filename}

## Import python files after copying them from Google Colab

In [5]:
init_file_struct()
from utils import *
from image_preprocessing import *

## Copy desired dataset to be used

In [7]:
# # copy_dataset('original.zip')
# # copy_dataset('clahe_2.zip')
# copy_dataset('clahe_10.zip')
copy_dataset('cropped.zip')

# SETUP

In [ ]:
# Functions used to preprocess images according to ImageNet 
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet50_preprocess
from tensorflow.keras.applications.densenet import preprocess_input as densenet_preprocess

In [36]:
# DATAFRAME
BODY_PART = 'ALL' # Which body part to use, can be "ALL" for full dataset
# IMAGE AUGMENTATION
ROTATION_R = 20 # Rotation range
W_SHIFT_R = 0.05 # Width shift range
H_SHIFT_R = 0.05 # Height shift range
BRIGHTNESS_R = (0.9, 1.1) # Brightness range
ZOOM_R = 0.1 # Zoom range
H_FLIP = True # Flip the image horizontally
PRE_FUNC = rescale # Preprocessing function to be applied to every image before feeding it to the model
# DATAFRAME FLOW
FLOW_DIR = CLAHE_10_DIR # Directory containing dataset images, choose one from filepath constants or use your own
BATCH_SIZE = 32 # Number of images per batch
IMAGE_SIZE = (224, 224) # Resize all images to this size
# MODEL
BASE_NAME = 'ResNet50' # Name corresponding to one of the architectures from Keras functional API
WEIGHTS = 'imagenet' # ImageNet pre-trained weights or a path to stored model weights
INPUT_SHAPE = (224, 224, 3) # Model input shape
POOLING = 'avg' # Pooling layer used after the last convolutional layer, can be None
OPTIMIZER = Adam(learning_rate=0.0001) # Optimzer used during training
MODEL_NAME = BASE_NAME + '_tmp' # Name used for storing model weights during and after training
# TRAINING
EPOCHS = 10 # Number of epochs for trainig

# Create datagens and flows

In [37]:
# Create dataframes from train valid split
train_df = get_dataframe(body_part=BODY_PART, split='train', path=DATAFRAME_PATH)
valid_df = get_dataframe(body_part=BODY_PART, split='valid', path=DATAFRAME_PATH)

# Create ImageDataGenerators, train_gen uses specified online augmentation, valid_gen only preprocesses images
train_gen, valid_gen = create_generators(rotation_r=ROTATION_R,
                                         w_shift_r=W_SHIFT_R,
                                         h_shift_r=H_SHIFT_R,
                                         brightness_r=BRIGHTNESS_R,
                                         zoom_r=ZOOM_R,
                                         h_flip=H_FLIP,
                                         pre_func=PRE_FUNC)
# Create dataframe flows, filepaths in dataframes are not absolute, so set directory parameter correctly
train_flow, valid_flow = create_dataframe_flows(train_gen=train_gen,
                                                valid_gen=valid_gen,
                                                train_df=train_df,
                                                valid_df=valid_df,
                                                directory=FLOW_DIR,
                                                img_size=IMAGE_SIZE,
                                                batch_size=BATCH_SIZE)

Found 36260 validated image filenames belonging to 2 classes.
Found 3197 validated image filenames belonging to 2 classes.


# Build and compile model

In [38]:
# Build and compile model
model = build_model(base_name=BASE_NAME,
                    weights=WEIGHTS, 
                    shape=INPUT_SHAPE,
                    pooling=POOLING,
                    optimizer=OPTIMIZER,
                    name=MODEL_NAME,
                    add_top=False,
                    metrics=[CohenKappa(num_classes=2, name='kappa')])

# Training

In [39]:
# Make checkpoint of weights from best epoch
best_path = CHECKPOINT_DIR + MODEL_NAME + '_best.h5'
checkpoint_best = ModelCheckpoint(filepath=best_path,
                                  save_weights_only=True,
                                  monitor='val_kappa',
                                  mode='max',
                                  save_best_only=True,
                                  verbose=1)

# Optional - make checkpoint of weights every epoch in case Google Colab runtime disconnects
# last_path = CHECKPOINT_DIR + MODEL_NAME + '_last.h5'
# checkpoint_last = ModelCheckpoint(filepath=last_path,
#                                   save_weights_only=True,
#                                   verbose=0)

# Early stopping
early_stop = EarlyStopping(monitor='val_kappa',
                           mode='max',
                           min_delta=0,
                           patience=5,
                           verbose=1,
                           restore_best_weights=True)

hist = model.fit(x=train_flow,
                 epochs=EPOCHS,
                 validation_data=valid_flow,
                 class_weight=get_class_weights(train_df),
                 verbose=1,
                 callbacks=[checkpoint_best, early_stop])

Epoch 1/10
1134/1134 [==============================] - ETA: 0s - loss: 0.2623 - kappa: 0.4587
Epoch 1: val_kappa improved from -inf to 0.36642, saving model to /content/drive/MyDrive/MURA/models/ResNet50_tmp_best.h5
1134/1134 [==============================] - 781s 683ms/step - loss: 0.2623 - kappa: 0.4587 - val_loss: 0.6242 - val_kappa: 0.3664
Epoch 2/10
1134/1134 [==============================] - ETA: 0s - loss: 0.2298 - kappa: 0.5516
Epoch 2: val_kappa improved from 0.36642 to 0.48689, saving model to /content/drive/MyDrive/MURA/models/ResNet50_tmp_best.h5
1134/1134 [==============================] - 783s 690ms/step - loss: 0.2298 - kappa: 0.5516 - val_loss: 0.5278 - val_kappa: 0.4869
Epoch 3/10
1134/1134 [==============================] - ETA: 0s - loss: 0.2193 - kappa: 0.5815
Epoch 3: val_kappa improved from 0.48689 to 0.57260, saving model to /content/drive/MyDrive/MURA/models/ResNet50_tmp_best.h5
1134/1134 [==============================] - 786s 693ms/step - loss: 0.2193 - kap

## Save last epoch weights and training history

In [40]:
# Optional save last weights / full model
model.save_weights(CHECKPOINT_DIR + MODEL_NAME + '_last.h5')
# model.save(CHECKPOINT_DIR + MODEL_NAME)

# Save training history
with open(CHECKPOINT_DIR + MODEL_NAME + '_hist', 'wb') as file_pi:
        pickle.dump(hist.history, file_pi)

# Evaluation
Evaluate trained model on test set, you can also load weights from previous runs to be evaluated

In [57]:
# Get test set
test_df = get_dataframe(body_part='ALL', split='test', path=DATAFRAME_PATH)

# Create dataframe flow for testing set (without augmentation)
test_flow = valid_gen.flow_from_dataframe(dataframe=test_df,
                                          directory=FLOW_DIR,
                                          x_col='filepath',
                                          y_col='label',
                                          class_mode='binary',
                                          target_size=IMAGE_SIZE,
                                          batch_size=BATCH_SIZE,
                                          seed=27,
                                          shuffle=False) # Dont shuffle so we can pair images with their filepaths

# Optional load weights from previous training
# model.load_weights('/content/drive/MyDrive/MURA/models/ResNet50_tmp_best.h5')
model.evaluate(x=test_flow, verbose=1)

Found 548 validated image filenames belonging to 2 classes.
18/18 [==============================] - 4s 197ms/step - loss: 0.4490 - kappa: 0.6176


[0.44896605610847473, 0.6176040172576904]